In [0]:
#define widgets -- NEEDS TO DEFINE IT ONCE
#dbutils.widgets.text("environment", "","")

#dynamic variables (pass it from ADF)
environment = dbutils.widgets.get("environment")

In [0]:
#============================= Global Variable Declaration ===================================#
from datetime import datetime
import json

bronze_folder_path = "/mnt/"+ environment + "/bronze"
silver_folder_path = "/mnt/"+ environment + "/silver"
gold_folder_path = "/mnt/"+ environment + "/gold"

In [0]:
%run ../bi_config/lib_helper

In [0]:
%run ../bi_config/lib_sentry

In [0]:
# ===============================Initialize JobRunInfo Table===========================================#
from datetime import datetime
from delta.tables import *
from pyspark.sql.types import TimestampType, LongType 
from pyspark.sql.functions import col, when, lit
import json

notebook = os.path.basename(getNotebookPath())

job_info = [{
    "job_name" : "ibi",
    "batch_id": datetime.strftime(datetime.utcnow(),'%Y%m%d%H%M%S'),
    "batch_start_datetime": datetime.strftime(datetime.utcnow(),'%Y-%m-%d %H:%M:%SZ'),
    "start_datetime": datetime.strftime(datetime.utcnow(),'%Y-%m-%d %H:%M:%S'),
    "status": "running"
  }]

def insert_jobinfo(job_info): 
  #job_info_obj = []
  job_info = spark.createDataFrame(job_info) 
  job_info = job_info.withColumn("job_name", job_info["job_name"])
  job_info = job_info.withColumn("batch_id",  job_info["batch_id"].cast(LongType()))
  job_info = job_info.withColumn("batch_start_datetime", job_info["batch_start_datetime"].cast(TimestampType()))
  job_info = job_info.withColumn("start_datetime", job_info["start_datetime"].cast(TimestampType()))
  job_info = job_info.withColumn("end_datetime", lit("").cast(TimestampType()))
  
  if max_last_updated_datetime["max(last_updated_datetime)"] == None:
     job_info = job_info.withColumn("last_updated_datetime",job_info["batch_start_datetime"].cast(TimestampType()))
  else: 
     job_info = job_info.withColumn("last_updated_datetime",lit(max_last_updated_datetime["max(last_updated_datetime)"]))

  job_info = job_info.withColumn("status", job_info["status"])
  job_info.write.format("delta").mode("append").save(job_info_table_path)
  
try:
  job_info_table_path = gold_folder_path + "/jobruninfo"
  df_job_info = spark.read.format("delta").load(job_info_table_path)

  max_last_updated_datetime = df_job_info.agg({"last_updated_datetime" : "max"}).collect()[0]
   
  insert_jobinfo(job_info)  
                                                                                 
  df_job_info_full = DeltaTable.forPath(spark, job_info_table_path)
  df_job_info = spark.read.format("delta").load(job_info_table_path)
  #df_job_info = df_job_info.filter(df_job_info.job_name == notebook) 
  #df_job_info.show()
  
    
except Exception as error:
  log_error("{} {}".format(notebook, error)) #log error in sentry
  raise dbutils.notebook.exit(error) #raise the exception